In [ ]:
!pip install tensorflow_hub

In [ ]:
!pip install tensorflow_text

In [ ]:
import numpy as np
import pandas as pd
import sklearn.preprocessing 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import string
import nltk
from nltk.corpus import stopwords
from textblob import Word
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import wordcloud
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split 
from keras.utils import np_utils
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
sns.set()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TUKL summer internship/week 2/Supreme Court Data 5.csv')
df.head()

,Case Files,Label,Category
0,2017 P T D 1481\n[Supreme Court of Pakistan]\n...,Non-Bailable,Against Federation
1,2015 S C M R 365\n[Supreme Court of Pakistan]\...,Bailable,illegal appointments
2,2016 S C M R 1420\n[Supreme Court of Pakistan]...,Bailable,Theft/Fraud
3,2016 S C M R 662\n[Supreme Court of Pakistan]\...,Bailable,Theft/Fraud
4,P L D 2012 Supreme Court 649\nPresent: Mian Sh...,Bailable,illegal appointments


In [ ]:
df['spam']=df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df.head()

,Case Files,Label,Category,spam
0,2017 P T D 1481\n[Supreme Court of Pakistan]\n...,Non-Bailable,Against Federation,0
1,2015 S C M R 365\n[Supreme Court of Pakistan]\...,Bailable,illegal appointments,0
2,2016 S C M R 1420\n[Supreme Court of Pakistan]...,Bailable,Theft/Fraud,0
3,2016 S C M R 662\n[Supreme Court of Pakistan]\...,Bailable,Theft/Fraud,0
4,P L D 2012 Supreme Court 649\nPresent: Mian Sh...,Bailable,illegal appointments,0


In [ ]:
df1 = df[['Case Files','Label']]
df1.isnull().sum()

Case Files    0
Label         0
dtype: int64

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [ ]:
def cleaning(df, stop_words):

    df['Case Files'] = df['Case Files'].apply(lambda x: ' '.join(x.lower() for x in x.split())) 

                               

    # Replacing the digits/numbers

    df['Case Files'] = df['Case Files'].str.replace('d', '')

            

    # Removing stop words

    df['Case Files'] = df['Case Files'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop_words))

    #lower casing
    df['Case Files'] = df['Case Files'].str.lower()

    #punctuation removal         
    df["Case Files"] = df["Case Files"].apply(lambda text: remove_punctuation(text))
    # Lemmatization

    df['Case Files'] = df['Case Files'].apply(lambda x: ' '.join([Word(x).lemmatize() for x in x.split()]))

            

    return df

stop_words = stopwords.words('english')

df1 = cleaning(df1, stop_words)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [ ]:
df2 = df1.rename(columns={'Case Files': 'Case_Files'})
df2.head()

,Case_Files,Label
0,2017 p 1481 supreme court pakistan present mia...,Non-Bailable
1,2015 c r 365 supreme court pakistan present na...,Bailable
2,2016 c r 1420 supreme court pakistan present a...,Bailable
3,2016 c r 662 supreme court pakistan present an...,Bailable
4,p l 2012 supreme court 649 present mian shakir...,Bailable


In [ ]:
tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(df2['Case_Files'].values)
X = tokenizer.texts_to_sequences(df2['Case_Files'].values)
X = pad_sequences(X)

In [ ]:
df2['encoded']=df2['Label'].apply(lambda x: 1 if x=='Bailable' else 0)
df2.head()

,Case_Files,Label,encoded
0,2017 p 1481 supreme court pakistan present mia...,Non-Bailable,0
1,2015 c r 365 supreme court pakistan present na...,Bailable,1
2,2016 c r 1420 supreme court pakistan present a...,Bailable,1
3,2016 c r 662 supreme court pakistan present an...,Bailable,1
4,p l 2012 supreme court 649 present mian shakir...,Bailable,1


In [ ]:
df2['encoded']

0     0
1     1
2     1
3     1
4     1
     ..
61    1
62    0
63    0
64    1
65    1
Name: encoded, Length: 66, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split#stratify ensures balance between spam and no spam in test and train

X_train, X_test, y_train, y_test = train_test_split(df2['Case_Files'],df2['encoded'],test_size=0.2, stratify=df2['encoded'])

In [ ]:
y_test.value_counts()

1    9
0    5
Name: encoded, dtype: int64

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):#generates embedding vector for sentence
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [ ]:
#so far we have used sequential models but now we are using BERT which is a functional model.
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')#name can be anything #input shape variable #data type string
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'encoder_outputs':  109482241   keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [ ]:
len(X_train)

52

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=50,shuffle=True)

Epoch 1/50
2/2 [==============================] - 31s 6s/step - loss: 0.6557 - accuracy: 0.6538
Epoch 2/50
2/2 [==============================] - 16s 6s/step - loss: 0.6673 - accuracy: 0.6346
Epoch 3/50
2/2 [==============================] - 16s 6s/step - loss: 0.6303 - accuracy: 0.6538
Epoch 4/50
2/2 [==============================] - 16s 6s/step - loss: 0.6783 - accuracy: 0.6538
Epoch 5/50
2/2 [==============================] - 16s 6s/step - loss: 0.6607 - accuracy: 0.6538
Epoch 6/50
2/2 [==============================] - 16s 6s/step - loss: 0.6066 - accuracy: 0.6731
Epoch 7/50
2/2 [==============================] - 16s 6s/step - loss: 0.6401 - accuracy: 0.6538
Epoch 8/50
2/2 [==============================] - 16s 6s/step - loss: 0.6454 - accuracy: 0.6346
Epoch 9/50
2/2 [==============================] - 16s 6s/step - loss: 0.6637 - accuracy: 0.6731
Epoch 10/50
2/2 [==============================] - 16s 6s/step - loss: 0.6637 - accuracy: 0.6538
Epoch 11/50
2/2 [======================

In [ ]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 6s 6s/step - loss: 0.6519 - accuracy: 0.6429


[0.651892364025116, 0.6428571343421936]